In [ ]:
%run ../config/init.py

### Creating results folders

In [ ]:
data_dir = os.path.join(RESULTS, DATASET, 'alignments')
result_dir = working_dir(os.path.join(RESULTS, DATASET, 'dga'))
matrix_file = os.path.join(data_dir, 'ExonReads.tsv')
tpm_matrix_file = os.path.join(data_dir, 'ExonTPM.tsv')
sample_table_file = os.path.join(DATA, DATASET, 'sample_table.csv')
sample_table = pandas.read_csv(sample_table_file, keep_default_na=False)
sample_table.head()

### Creating comparisons
In this cell, an array with all combinations of **conditions** is created.  
 
If you just want to explore a set of comparisons remove this cell and add the **comparisons** list manually. 
```
comparisons = [
    ['cond1', 'cond2'],
    ['cond2', 'cond3']
]
``` 

In [ ]:
comparisons = []
for s in itertools.combinations(sample_table['condition'].unique(), 2):
    comparisons.append(list(s)) 
comparisons

### Executing Deseq2 and EdgeR workflows

In [ ]:
log_suffix = 'dga.log'

min_TPM = 1

cmd_deseq2_header = '{0} {1}/R/deseq2-2conditions.cwl --factor {2} --matrix {3} --tpm_matrix {4} --gene_column Gene_Chr_Start --sample_column sample_name --fc {5} --fdr {6} --min_tpm {7} '.format(
        CWLRUNNER, CWLTOOLS, sample_table_file, matrix_file, tpm_matrix_file, fc, fdr, min_TPM)
cmd_edgeR_header = '{0} {1}/R/edgeR-2conditions.cwl --factor {2} --matrix {3} --tpm_matrix {4} --gene_column Gene_Chr_Start --sample_column sample_name --fc {5} --fdr {6} --gene_length_column ExonLength  --min_tpm {7} '.format(
        CWLRUNNER, CWLTOOLS, sample_table_file, matrix_file, tpm_matrix_file, fc, fdr, min_TPM)

for c in comparisons:
    if not os.path.exists('condition_{0}_vs_{1}_deseq2.csv'.format(c[0], c[1])):
        log_file = '{0}_vs_{1}_deseq2_{2}'.format(c[0], c[1], log_suffix)
        cmd = '{0} --condition1 {1} --condition2 {2} > {3} 2>&1\n'.format(cmd_deseq2_header, c[0], c[1], log_file)
        run_command(cmd)
        check_cwl_command_log(log_file)
    if not os.path.exists('condition_{0}_vs_{1}_edgeR.csv'.format(c[0], c[1])):
        log_file = '{0}_vs_{1}_edge_{2}'.format(c[0], c[1], log_suffix)
        cmd = '{0} --condition1 {1} --condition2 {2} > {3} 2>&1\n'.format(cmd_edgeR_header, c[0], c[1], log_file)
        run_command(cmd)
        check_cwl_command_log(log_file)


### Creates TPM and reads matrices for genes

In [ ]:
log_suffix_union = 'union.log'
    
cmd_volcano_header = '{0} {1}/R/volcano_plot.cwl --fdr {2} --fc {3} '.format(CWLRUNNER, CWLTOOLS, fdr, fc)

cmd_heatmap_header = '{0} {1}/R/dga_heatmaps.cwl --factor {2} --matrix {3} --gene_column Gene_Chr_Start --sample_column SampleID --fdr {4} --fc {5}'.format(
        CWLRUNNER, CWLTOOLS, sample_table_file, matrix_file, fdr, fc)
    
log_suffix_union = 'union.log'
    
cmd_volcano_header = '{0} {1}/R/volcano_plot.cwl --fdr {2} --fc {3} '.format(CWLRUNNER, CWLTOOLS, fdr, fc)

cmd_heatmap_header = '{0} {1}/R/dga_heatmaps.cwl --factor {2} --matrix {3} --gene_column Gene_Chr_Start --sample_column sample_name --fdr {4} --fc {5}'.format(
        CWLRUNNER, CWLTOOLS, sample_table_file, matrix_file, fdr, fc)
    

for c in comparisons:
    deseq2_df = pandas.read_csv('condition_{0}_vs_{1}_deseq2.csv'.format(c[0], c[1]))
    edgeR_df = pandas.read_csv('condition_{0}_vs_{1}_edgeR.csv'.format(c[0], c[1]))
    df = edgeR_df.merge(deseq2_df, left_on='Gene_Chr_Start', right_on='Gene_Id')
    df = df.drop(['Gene_Chr_Start'], axis=1)
    union_df = df[(df['FDR'] <= fdr) & (abs(df['logFC']) >= fc) & (df['padj'] <= fdr) & (abs(df['log2FoldChange']) >= fc)]
    unifying_data = []
    for i, r in df.iterrows():
        if abs(r['logFC']) == r['logFC']:
            logFC = min(float(r['logFC']), float(r['log2FoldChange']))
        else:
            logFC = max(float(r['logFC']), float(r['log2FoldChange']))
        unifying_data.append({
            'Gene_Id': r['Gene_Id'],
            'FDR': max(float(r['FDR']), float(r['padj'])),
            'logFC': logFC
        })
    unifying_data_df = pandas.DataFrame(unifying_data)
    unifying_data_df = unifying_data_df[['Gene_Id', 'logFC', 'FDR']]
    if len(unifying_data_df) > 0:
        unifying_file = 'condition_{0}_vs_{1}_union.csv'.format(c[0], c[1])
        unifying_data_df.to_csv(unifying_file, index=None)

        over_df = unifying_data_df[(unifying_data_df['FDR'] <= fdr)&(unifying_data_df['logFC'] >= fc)]
        over_df.to_csv('condition_{0}_vs_{1}_union_over-expressed.csv'.format(c[0], c[1]), index=None)
        under_df = unifying_data_df[(unifying_data_df['FDR'] <= fdr)&(unifying_data_df['logFC'] <= -1.0 * fc)]
        under_df.to_csv('condition_{0}_vs_{1}_union_under-expressed.csv'.format(c[0], c[1]), index=None)

        print("Genes with FDR <= %.2f and logFC >= %.2f: %d" % (fdr, fc, len(over_df)))
        print("Genes with FDR <= %.2f and logFC <= -%.2f: %d" % (fdr, fc, len(under_df)))

        log_file = '{}_vs_{}_volcano_{}'.format(c[0], c[1], log_suffix_union)
        cmd = '{0} --data {1} --out condition_{2}_vs_{3}_union_volcano.pdf > {4} 2>&1\n'.format(
            cmd_volcano_header, unifying_file,c[0], c[1], log_file)
        run_command(cmd)
        check_cwl_command_log(log_file)
        
        log_file = '{}_vs_{}_heatmap_{}'.format(c[0], c[1], log_suffix_union)
        cmd = '{0} --dga_data {1} --out_expression {2} --out_correlation {3} --out_pca {4} > {5} 2>&1\n'.format(
            cmd_heatmap_header, unifying_file,
            'condition_{0}_vs_{1}_union_expression_heatmap.pdf'.format(c[0], c[1]),
            'condition_{0}_vs_{1}_union_correlation_heatmap.pdf'.format(c[0], c[1]),
            'condition_{0}_vs_{1}_union_pca.pdf'.format(c[0], c[1]),
            log_file)
        run_command(cmd)
        check_cwl_command_log(log_file)    